In [33]:
from math import *
import random
from random import randint

class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width,empty):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.empty     = empty
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}
        if empty==True:
            #on ajoute tous les voisins
            self.neighbors = {(i,j): {(i-1,j),(i,j-1),(i+1,j),(i,j+1)} for i in range(height) for j in range (width)}
            #on supprime les murs invalides
            for i in range(height):
                self.neighbors[i,width-1].remove((i,width))
            #on supprime les voisins qui n'apparaissent pas dans le labyrinthe
            for j in range(self.height):
                for k in range(self.width):
                    a_supprimer=[]
                    for l in self.neighbors[(j,k)]:
                        if l[0]>self.height or l[0]<0 or l[1]>self.width or l[1]<0 :
                            a_supprimer.append(l)
                    for m in range(len(a_supprimer)):
                            self.neighbors[(j,k)].remove(a_supprimer[m])

    def info(self):
        """
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = f"{self.height} x {self.width}\n"
        txt += str(self.neighbors)
        return txt

    def __str__(self):
        """
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    def overlay(self, content=None):
        """
        Rendu en mode texte, sur la sortie standard,
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        """
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            content = content | {(i, j): ' ' for i in range(
                self.height) for j in range(self.width) if (i,j) not in content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt
    
    def add_wall(self, c1, c2):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
            
    def remove_wall(self, c1:tuple, c2:tuple):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Suppression du mur
        if c2 not in self.neighbors[c1]:      # Si c2 n'est pas dans les voisines de c1
            self.neighbors[c1].add(c2) # on l'ajoute
        if c1 not in self.neighbors[c2]:      # Si c3 n'est pas dans les voisines de c2
            self.neighbors[c2].add(c1) # on l'ajoute
            
    def fill(self):
        #On vide le voisinnage pour ajouter tous les murs
        self.neighbors= {(i,j): set() for i in range(self.height) for j in range (self.width)}
        
    def empty_lab(self):
        #On remplis le voisinnage pour enlever tous les murs
        self.neighbors = {(i,j): {(i-1,j),(i,j-1),(i+1,j),(i,j+1)} for i in range(self.height) for j in range (self.width)}
        #on retire les murs invalides
        for i in range(self.height):
            self.neighbors[i,self.width-1].remove((i,self.width))
        # on retire les voisins qui n'apparaissent pas dans le labyrinthe
        for j in range(self.height):
            for k in range(self.width):
                a_supprimer=[]
                for l in self.neighbors[(j,k)]:
                    if l[0]>self.height or l[0]<0 or l[1]>self.width or l[1]<0 :
                        a_supprimer.append(l)
                for m in range(len(a_supprimer)):
                        self.neighbors[(j,k)].remove(a_supprimer[m])
            
            
    def get_walls(self):
        """
        Affichage des murs du labyrinthe sous forme de liste de tuples représentant les cellules entre lesquelles est le 
        mur
        Retour:
            liste_murs(list): Liste des murs
        """
        liste_murs=[]
        mur=[]
        #on parcourt les cases
        for i in range(self.height):
            for j in range(self.width):
                #si un voisin n'est pas présent dans la liste, c'est qu'il y a un mur, on l'ajoute donc
                if (i+1,j)not in self.neighbors[i,j] and (i+1<self.height):
                    mur=[(i,j),(i+1,j)]
                    #on vérifie que le mur ne soit pas deja dans la liste pour éviter les doublons
                    if mur not in liste_murs:
                        liste_murs.append(mur)
                if (i,j+1)not in self.neighbors[i,j] and (j+1<self.width):
                    mur=[(i,j),(i,j+1)]
                    if mur not in liste_murs:
                        liste_murs.append(mur)
        return liste_murs
    
    def get_contiguous_cells(self,c:tuple):
        """
        Affichage des cellules contigues a la cellule c
        Retour :
            liste_cont(list): liste de tuple des cellules collées a la cellule c
        """
        liste_cont=[(c[0]-1,c[1]),(c[0],c[1]+1),(c[0]+1,c[1]),(c[0],c[1]-1)]
        #on enlève les cases non contenues dans le labyrinthe
        to_remove=[]
        for i in liste_cont:
            if i[0]>self.height-1 or i[0]<0 or i[1]>self.width-1 or i[1]<0 :
                to_remove.append(i)
        for j in to_remove:
            liste_cont.remove(j)
        return liste_cont
    
    def get_reachable_cells(self,c:tuple):
        """
        Affichage des cellules atteignables depuis la cellule passée en paramètre
        Retour :
            liste_att(list): liste de tuple des cellules atteignables
        """
        liste_att=[]
        for i in self.neighbors[c]:
            liste_att.append(i)
        return liste_att
    
    
    @classmethod
    def gen_btree(cls,h:int,w:int):
        """
        Méthode de classe permettant de generer un labyrinthe à h lignes et w colonnes, en utilisant l’algorithme de 
        construction par arbre binaire.
        Retour :
            laby(Maze) : Le labyrinthe géneré
        """
        #on crée un labyrinthe plein
        laby=Maze(h,w,False)
        #on parcours les cases
        for i in range(laby.height):
            for j in range(laby.width):
                #on choisis une direction au hasard
                n=randint(0,1)
                #on supprime le mur choisi
                if n==0:
                    #on vérifie que le mur existe
                    if [(i,j),(i,j+1)] in laby.get_walls() or [(i,j+1),(i,j)] in laby.get_walls():
                        #on le supprime si oui
                        laby.remove_wall((i,j),(i,j+1))
                        laby.remove_wall((i,j+1),(i,j))
                    #sinon on supprime l'autre mur, s'il existe
                    elif [(i,j),(i+1,j)] in laby.get_walls() or [(i+1,j),(i,j)] in laby.get_walls():
                        laby.remove_wall((i,j),(i+1,j))
                        laby.remove_wall((i+1,j),(i,j))
                else :
                    if  [(i,j),(i+1,j)] in laby.get_walls() or [(i+1,j),(i,j)] in laby.get_walls():
                        laby.remove_wall((i,j),(i+1,j))
                        laby.remove_wall((i+1,j),(i,j))
                    elif [(i,j),(i,j+1)] in laby.get_walls() or [(i,j+1),(i,j)] in laby.get_walls():
                        laby.remove_wall((i,j),(i,j+1))
                        laby.remove_wall((i,j+1),(i,j))
        return laby
    
    @classmethod
    def gen_sidewinder(cls,h:int,w:int):
        """
        Méthode de classe permettant de generer un labyrinthe à h lignes et w colonnes, en utilisant l’algorithme de 
        construction sidewinder.
        Retour :
            laby(Maze) : Le labyrinthe géneré
        """
        #on crée un labyrinthe plein
        laby=Maze(h,w,False)
        #on parcourt les cases
        for i in range(0,laby.height-1):
            seq=[]
            for j in range(0,laby.width-1):
                seq.append((i,j))
                #on tire a pile ou face
                n=randint(0,1)
                #pile
                if n==0:
                    #on vérifie que le mur existe
                    if [(i,j),(i,j+1)] in laby.get_walls() or [(i,j+1),(i,j)] in laby.get_walls():
                        #on le supprime si oui
                        laby.remove_wall((i,j),(i,j+1))
                        laby.remove_wall((i,j+1),(i,j))
                #face
                else:
                    #on choisit une cellule au hasard
                    m=randint(0,len(seq)-1)
                    k=seq[m][0]
                    l=seq[m][1]
                    #on casse son mur SUD
                    if  [(k,l),(k+1,l)] in laby.get_walls() or [(k+1,l),(k,l)] in laby.get_walls():
                        laby.remove_wall((k,l),(k+1,l))
                        laby.remove_wall((k+1,l),(k,l))
                    #on reinitialise la séquence
                    seq=[]
            seq.append((i,laby.width-1))
            #on choisit une cellule au hasard
            m=randint(0,len(seq)-1)
            k=seq[m][0]
            l=seq[m][1]
            #on casse son mur SUD
            if  [(k,l),(k+1,l)] in laby.get_walls() or [(k+1,l),(k,l)] in laby.get_walls():
                laby.remove_wall((k,l),(k+1,l))
                laby.remove_wall((k+1,l),(k,l))
        #on casse les murs EST de la dernière ligne
        for k in range (laby.width):
            #on vérifie que le mur existe
                if [(laby.height-1,k),(laby.height-1,k+1)] in laby.get_walls() or [(laby.height-1,k+1),(laby.height-1,k)] in laby.get_walls():
                    #on le supprime si oui
                    laby.remove_wall((laby.height-1,k),(laby.height-1,k+1))
                    laby.remove_wall((laby.height-1,k+1),(laby.height-1,k))
        return laby
    
    @classmethod
    def gen_fusion(cls,h:int,w:int):
        """
        Méthode de classe permettant de generer un labyrinthe à h lignes et w colonnes, en utilisant l’algorithme de 
        construction par fusion de chemins.
        Retour :
            laby(Maze) : Le labyrinthe géneré
        """
         #on crée un labyrinthe plein
        laby=Maze(h,w,False)
        label={}
        n=1
        #on parcourt les cases
        for i in range(h):
            for j in range(w):
                label[(i,j)]=n
                n+=1
            murs=laby.get_walls()
            random.shuffle(murs)
        for j in murs:
            labj0=label[j[0]]
            if label[j[0]]!=label[j[1]]:
                laby.remove_wall(j[0],j[1])
                laby.remove_wall(j[1],j[0])
                for l in label:
                    if label[l]==label[j[1]]:
                        label[l]=label[j[0]]
                    print(label)
            print("")
        return laby
    

    @classmethod
    def gen_exploration(cls,h:int,w:int):
        """
        Méthode de classe permettant de generer un labyrinthe à h lignes et w colonnes, en utilisant l’algorithme
        d’exploration exhaustive.
        Retour :
            laby(Maze) : Le labyrinthe géneré
        """
        #on crée un labyrinthe plein
        laby=Maze(h,w,False)
        #on crée le dictionnaire des cellules visitée, pour une cellule de coordonnées (i,j), la valeur 0 veut dire non visité, la valeur 1 veut dire visité
        #on initialise toutes les cellules a 0, non visitées
        visit={}
        for i in range(h):
            for j in range(w):
                visit[(i,j)]=0
        #on crée une pile vide
        pile=[]
        #on choisit une cellule au hasard
        m=(randint(0,h-1),randint(0,w-1))
        #on la marque comme visitée
        visit[m]=1
        #on l'ajoute a la pile
        pile.append(m)
        #tant que la pile n'est pas vide :
        while len(pile) > 0:
            #on retire la cellule au sommet de la pile
            top_pile=pile[-1]
            pile.pop()
            #si elle a des voisins non visités :
            vois_non_vis=[]
            for k in laby.get_contiguous_cells(top_pile):
                if visit[k]==0:
                    vois_non_vis.append(k)
            if  len(vois_non_vis)>0:
                #on remet la cellule sur la pile
                pile.append(top_pile)
                #on choisis une voisine au hasard
                rand_vois=vois_non_vis[randint(0,len(vois_non_vis)-1)]
                #on casse le mur qui les relie
                laby.remove_wall(top_pile,rand_vois)
                #on marque la cellule choisie comme visitée
                visit[rand_vois]=1
                #on la met sur la pile
                pile.append(rand_vois)
        return laby
    
    @classmethod
    def gen_wilson(cls,h:int,w:int):
        """
        Méthode de classe permettant de generer un labyrinthe à h lignes et w colonnes, en utilisant l’algorithme
        de Wilson.
        Retour :
            laby(Maze) : Le labyrinthe géneré
        """
         #on crée un labyrinthe plein
        laby=Maze(h,w,False)
        #on crée le dictionnaire des cellules marqué, pour une cellule de coordonnées (i,j), la valeur 0 veut dire non marqué la valeur 1 veut dire marqué
        #on initialise toutes les cellules a 0, non marquées
        marque={}
        for i in range(h):
            for j in range(w):
                marque[(i,j)]=0
        #on choisit une cellule au hasard
        m=(randint(0,h-1),randint(0,w-1))
        #on la marque
        marque[m]=1
        #tant qu'il reste des cellules non marquées :
        while 0 in marque.values():
            #Choisir une cellule de départ au hasard, parmi les cellules non marquées
            non_marque=[]
            for i in marque:
                if marque[i]==0:
                    non_marque.append(i)
            cell_dep=non_marque[randint(0,len(non_marque)-1)]
            chemin=[cell_dep]
            #Effectuer une marche aléatoire jusqu’à ce qu’une cellule marquée soit atteinte
            while marque[cell_dep]==0:
                choice=laby.get_contiguous_cells(cell_dep)
                current=cell_dep
                cell_dep=choice[randint(0,len(choice)-1)]
                if cell_dep in chemin:
                    while chemin[-1]!=cell_dep:
                        chemin.pop()
                chemin.append(cell_dep)
            #Marquer chaque cellule du chemin
            for i in chemin:
                marque[i]=1
            #Casser tous les murs rencontrés jusqu'a la cellule marquée
            for i in range (0,len(chemin)-1):
                laby.remove_wall(chemin[i],chemin[i+1])
        return laby
    
    def solve_dfs(self,start:tuple,stop:tuple)->list:
        """
        Méthode d'instance permettant de résoudre un labyrinthe en allant de la case start a la case stop, en utilisant la résolution par parcours en longueur
        Retour :
            chemin(list) : Le chemin le plus court de start a stop
        """
        pile=[start]
        
            
#laby=Maze(4,4,False)
#laby=Maze.gen_btree(4, 4)
#laby=Maze.gen_sidewinder(15,15)
#laby=Maze.gen_fusion(15,15)
#laby=Maze.gen_exploration(12,12)
laby=Maze.gen_wilson(12, 12)
print(laby)
#laby.empty_lab()
#laby.add_wall((0, 0), (0, 1))
#laby.add_wall((0, 1), (1, 1))
#print(laby.info())
#print(laby.get_walls())
#print(laby.get_contiguous_cells((0,1)))
#print(laby.get_reachable_cells((0,1)))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ c   o ┃                   ┃                   ┃
┣━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━┫
┃     u ┃   ┃   ┃   ┃       ┃   ┃   ┃       ┃   ┃
┣━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ┫
┃     c   o     ┃                               ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃   ┃     u ┃ !     ┃       ┃   ┃       ┃   ┃   ┃
┣━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃               ┃   ┃       ┃   ┃           ┃   ┃
┣━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━┫
┃       ┃       ┃   ┃           ┃   ┃       ┃   ┃
┣━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ┫
┃       ┃       ┃           ┃       ┃           ┃
┣━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ┫
┃       ┃       ┃                           ┃   ┃
┣━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ┫
┃           ┃       ┃   ┃       ┃   ┃       ┃   ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃   ┃               ┃       ┃           ┃       ┃
